# Building LLM applications: Notebook 01

# Langchain basics

## Initialize

In [1]:
import os
import dotenv

from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [2]:
MODEL = 'llama3.2:3b-instruct-fp16'

In [3]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

Using Ollama server: http://kqrw311-g5-12xlarge-a.img.astrazeneca.net:8080


## Excercise 01: Connect to an LLM

Connect to an LLM and make a simple request (e..g translate a senetence)

```
Translate to French: I like coffee.
```

Take a look at the LLM parameters, and full response 

In [4]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

messages = "Translate to French: I like coffee."

# Invoke the LLM
ret = llm.invoke(messages)
ret.pretty_print()

================================== Ai Message ==================================

I would say:

J'aime le café.

Here's a breakdown of the translation:

* "I" is translated to "j'ai", which is the first person singular form of the verb "avoir" (to have).
* "like" is translated to "aime", which is the first person singular form of the verb "aimer" (to love or like).
* "coffee" remains the same, as it's a common noun that doesn't change much in French.

Note: You can also say "Je bois du café" if you want to express your preference for coffee. This means "I drink coffee".


In [5]:
# Show details of the response
print(ret.model_dump_json(indent=2))

{
  "content": "I would say:\n\nJ'aime le café.\n\nHere's a breakdown of the translation:\n\n* \"I\" is translated to \"j'ai\", which is the first person singular form of the verb \"avoir\" (to have).\n* \"like\" is translated to \"aime\", which is the first person singular form of the verb \"aimer\" (to love or like).\n* \"coffee\" remains the same, as it's a common noun that doesn't change much in French.\n\nNote: You can also say \"Je bois du café\" if you want to express your preference for coffee. This means \"I drink coffee\".",
  "additional_kwargs": {},
  "response_metadata": {
    "model": "llama3.2:3b-instruct-fp16",
    "created_at": "2025-01-16T20:45:36.166330257Z",
    "done": true,
    "done_reason": "stop",
    "total_duration": 1235155190,
    "load_duration": 22510415,
    "prompt_eval_count": 33,
    "prompt_eval_duration": 8000000,
    "eval_count": 130,
    "eval_duration": 1203000000,
    "message": {
      "role": "assistant",
      "content": "",
      "images": 

## Excercise 02: Prompt template

In [6]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt from a template
prompt_template = """
Translate to French:

{sentence}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)


# Create a "chain" of runnable objects
runnable = prompt | llm

# Invoke the 'runnable'
ret = runnable.invoke({"sentence": "I like coffee."})
ret.pretty_print()

================================== Ai Message ==================================

J'aime le café.


## Excercise 03: Streaming

In [7]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt from a template
prompt_template = """
Translate to French:

{sentence}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)


# Create a "chain" of runnable objects
runnable = prompt | llm

# Invoke the 'runnable' as a stream, print each block as it arrives
blocks = []
for b in runnable.stream({"sentence": "I like coffee."}):
    print(b.content, end='')
    blocks.append(b)

print(f"\n----\nReceived {len(blocks)} blocks")

J'apprécie le café.
----
Received 10 blocks


In [8]:
# What does a block look like
print(blocks[0])

content='J' additional_kwargs={} response_metadata={} id='run-1a23fc4e-9a6f-487d-928e-5e6b70f89095'


## Excercise 04: Roles

In [9]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

messages = [
    ('system', "You are helping to translate the user's inputs from english to french"),
    ('human', "I like coffee")
]

ret = llm.invoke(messages)
ret.pretty_print()

================================== Ai Message ==================================

J'adore le café.


## Excercise 04: Message prompt templates with roles

In [10]:
# Create the LLM object
messages_template = [
    ('system', "You are helping to translate the user's inputs from English to {language}"),
    ('human', "{sentence}")
]

llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)
prompt = ChatPromptTemplate.from_messages(messages=messages_template)
runnable = prompt | llm

# Run the 'runnable'
ret = runnable.invoke({"language": "French", "sentence": "I like coffee."})
ret.pretty_print()

================================== Ai Message ==================================

J'aime le café.
